In [5]:
# CÓDIGO A EJECUTAR EN LA CELDA 1

import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from sklearn.ensemble import RandomForestClassifier

# URL base de la liga chilena 2024
URL_LIGA_CHILENA_2024 = 'https://www.flashscore.com/football/chile/liga-de-primera-2024/results/'

print("Configuración y librerías cargadas. El entorno está listo.")

Configuración y librerías cargadas. El entorno está listo.


In [6]:
# CÓDIGO A EJECUTAR EN LA CELDA 2

def obtener_urls_partidos_flashscore(url_liga_general):
    """
    Usa Selenium para cargar la tabla dinámica y extraer la URL larga de cada partido.
    """
    print("Iniciando Selenium para cargar la tabla dinámica (Fase 1)...")
    
    options = Options()
    options.add_argument('--headless') # Modo sin ventana
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install()) # Gestión automática de driver
    driver = webdriver.Chrome(service=service, options=options) 
    
    driver.get(url_liga_general)
    driver.implicitly_wait(10)
    
    # Intenta cargar todos los partidos haciendo clic en 'Mostrar más'
    try:
        mostrar_mas_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'event__more'))
        )
        while True:
            try:
                mostrar_mas_button.click()
                time.sleep(2) 
            except:
                break
    except:
        pass
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit() 
    
    live_table = soup.find('div', id='live-table')
    if not live_table: 
        print("Error: La tabla #live-table no se cargó correctamente.")
        return []

    filas_partidos = live_table.find_all('div', class_=lambda x: x and 'event__match' in x)
    lista_urls = []
    base_url = 'https://www.flashscore.com'
    
    for fila in filas_partidos:
        try:
            # Buscamos el enlace largo (<a>) con la clase 'eventRowLink'
            enlace_tag = fila.find('a', href=True, class_='eventRowLink') 

            if enlace_tag and 'href' in enlace_tag.attrs:
                slug_completo = enlace_tag['href']
                
                # Eliminamos el token MID (?mid=...) para obtener la URL limpia y estable
                if '?' in slug_completo:
                    slug_completo = slug_completo.split('?')[0]
                
                url_partido = f"{base_url}{slug_completo}"
                
                lista_urls.append(url_partido)
            
        except Exception:
            continue
            
    print(f"Se recolectaron {len(lista_urls)} URLs de partidos (formato largo).")
    return lista_urls

# Ejecución de la Fase 1
partidos_urls = obtener_urls_partidos_flashscore(URL_LIGA_CHILENA_2024)

Iniciando Selenium para cargar la tabla dinámica (Fase 1)...
Se recolectaron 104 URLs de partidos (formato largo).


In [ ]:
# CÓDIGO A EJECUTAR EN LA CELDA 3

def extraer_estadisticas_detalladas_selenium(url_base):
    """
    Usa Selenium para cargar el contenido dinámico y extraer todas las estadísticas.
    Incluye Anti-Bot measures y Tolerancia a Fallos.
    """
    # Construimos la URL final para ir a la pestaña de estadísticas
    if not url_base.endswith('/'):
        url_base += '/' 
    # Añadimos la ruta de la pestaña de estadísticas directamente
    url_stats = f"{url_base}estadisticas-del-partido/" 
    datos = {'URL': url_stats}
    
    # 1. Inicializar el driver con opciones Anti-Bot y Headless
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    datos['Local'], datos['Visitante'] = None, None
    datos['Goles_Local'], datos['Goles_Visitante'] = None, None
    
    try:
        driver.get(url_stats)
        
        # 2. Esperar a que el botón STATS esté visible (o la tabla cargada)
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.tabContent_match-statistics')))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # --- A. Extracción de Nombres y Resultado (BLOQUE TOLERANTE) ---
        try:
            # Selectores de Cabecera confirmados
            datos['Local'] = soup.find('div', class_=lambda x: x and 'duelParticipant__home' in x).text.strip()
            datos['Visitante'] = soup.find('div', class_=lambda x: x and 'duelParticipant__away' in x).text.strip()
            
            score_wrapper = soup.find('div', class_='duelParticipant__score').find('div', class_='detailScore__wrapper')
            score_tags = score_wrapper.find_all('span')
            
            datos['Goles_Local'] = int(score_tags[0].text)
            datos['Goles_Visitante'] = int(score_tags[2].text)
            
        except Exception:
            pass # Continuamos, dejando Goles como None
            
        if datos['Local'] is None:
            return None # Fila inútil sin nombre
            
        # --- B. Extracción de la Tabla de Estadísticas (CONFIRMADO) ---
        stats_container = soup.find('div', class_='tabContent_match-statistics')
        
        if stats_container:
            # Buscamos filas por patrón de clase dinámica 'wcl-row'
            stat_rows = stats_container.find_all('div', class_=lambda x: x and re.search(r'wcl-row', x))
            
            for row in stat_rows:
                try:
                    valor_local_tag = row.find('div', class_=lambda x: x and 'wcl-homeValue' in x)
                    nombre_stat_tag = row.find('div', class_=lambda x: x and 'wcl-stat-category' in x)
                    valor_visitante_tag = row.find('div', class_=lambda x: x and 'wcl-awayValue' in x)
                    
                    if valor_local_tag and nombre_stat_tag and valor_visitante_tag:
                        nombre = nombre_stat_tag.text.strip().replace(' ', '_').replace('/', '_').lower()
                        
                        # Los valores están dentro de un strong/span, extraemos el texto
                        valor_local = valor_local_tag.text.strip()
                        valor_visitante = valor_visitante_tag.text.strip()

                        # Función para limpiar el valor (solo números y punto)
                        def limpiar_valor(valor):
                            return re.sub(r'[^0-9.]', '', valor)

                        valor_local = limpiar_valor(valor_local)
                        valor_visitante = limpiar_valor(valor_visitante)
                        
                        datos[f'{nombre}_local'] = float(valor_local) if valor_local else 0
                        datos[f'{nombre}_visitante'] = float(valor_visitante) if valor_visitante else 0

                except Exception:
                    continue
        
        datos.setdefault('posesion_del_balon_local', None)
        datos.setdefault('tiros_al_arco_local', None)

        return datos

    except Exception:
        return None
        
    finally:
        driver.quit()

In [ ]:
# CÓDIGO A EJECUTAR EN LA CELDA 4

def crear_dataset_completo(lista_urls):
    datos_totales = []
    
    print(f"Iniciando extracción detallada de {len(lista_urls)} partidos...")
    
    for i, url in enumerate(lista_urls):
        datos = extraer_estadisticas_detalladas_selenium(url) 
        
        if datos:
            datos_totales.append(datos)
        
        if (i + 1) % 20 == 0:
            print(f"Progreso: {i + 1}/{len(lista_urls)} partidos procesados. Datos recolectados: {len(datos_totales)}")
        
        time.sleep(2) 

    df_dataset_crudo = pd.DataFrame(datos_totales)
    return df_dataset_crudo

# --- EJECUCIÓN FINAL ---

if 'partidos_urls' in locals() and partidos_urls:
    
    df_dataset_crudo = crear_dataset_completo(partidos_urls)

    print("\n" + "=" * 50)
    print(f"✅ Extracción Finalizada. Total de partidos con datos: {len(df_dataset_crudo)}")
    print("=" * 50)
    
    display(df_dataset_crudo.head())
    df_dataset_crudo.to_csv('dataset_flashscore_completo_crudo.csv', index=False)
    
else:
    print("❌ Error: La lista 'partidos_urls' está vacía. Revisa la Celda 2.")